# Analysis :

## Imports

In [1]:
#imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import dateutil
%matplotlib inline


# Keras imports

In [2]:
from sklearn.metrics import confusion_matrix, precision_score
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.regularizers import l2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from keras.models import load_model

Using TensorFlow backend.


## Data loading

In [3]:
#reading file
df = pd.read_excel("Load Survey Data (6).xlsx")

## Data PreProcessing and cleaning

In [4]:
#converting timestamp oject to datetime object
df['Timestamp'] = pd.to_datetime(df.Timestamp)

#list of columns to drop
drop_columns = ['Vr (Volt)','Vy (Volt)','Vb (Volt)','Ir (Amp)','Iy (Amp)','Ib (Amp)','Freq (Hz)']

#dropped columns dataframe
df = df.drop(drop_columns,axis=1)

#changing column names
df.columns = ['CCMS','timestamp','kwh','kvah','pf_total','total_load']
df['time_hour'] = df.timestamp.apply(lambda x: x.hour)

In [5]:
df.columns

Index(['CCMS', 'timestamp', 'kwh', 'kvah', 'pf_total', 'total_load',
       'time_hour'],
      dtype='object')

In [6]:
df.dtypes

CCMS                  object
timestamp     datetime64[ns]
kwh                  float64
kvah                 float64
pf_total              object
total_load           float64
time_hour              int64
dtype: object

In [7]:
df.head()

,CCMS,timestamp,kwh,kvah,pf_total,total_load,time_hour
0,SMR03-0219-0252,2019-05-01 23:00:00,2.732,2.752,0.955 Ld,5.17,23
1,SMR03-0219-0252,2019-05-01 22:00:00,5.108,5.143,0.987 Ld,5.17,22
2,SMR03-0219-0252,2019-05-01 21:00:00,5.120,5.156,0.987 Ld,5.17,21
3,SMR03-0219-0252,2019-05-01 20:00:00,4.985,5.025,0.985 Ld,5.17,20
4,SMR03-0219-0252,2019-05-01 19:00:00,0.372,0.375,0.87 Lag,5.17,19


## Basic data description

In [8]:
df.describe()

,kwh,kvah,total_load,time_hour
count,742.000000,742.000000,7.420000e+02,742.000000
mean,2.139387,2.153534,5.170000e+00,11.505391
std,2.296270,2.311301,4.799399e-14,6.935356
min,0.000000,0.000000,5.170000e+00,0.000000
25%,0.000000,0.000000,5.170000e+00,5.250000
50%,0.480000,0.484000,5.170000e+00,12.000000
75%,4.493000,4.520000,5.170000e+00,17.750000
max,5.453000,5.489000,5.170000e+00,23.000000


In [9]:
print(df)

                CCMS           timestamp    kwh   kvah    pf_total  \
0    SMR03-0219-0252 2019-05-01 23:00:00  2.732  2.752   0.955  Ld   
1    SMR03-0219-0252 2019-05-01 22:00:00  5.108  5.143   0.987  Ld   
2    SMR03-0219-0252 2019-05-01 21:00:00  5.120  5.156   0.987  Ld   
3    SMR03-0219-0252 2019-05-01 20:00:00  4.985  5.025   0.985  Ld   
4    SMR03-0219-0252 2019-05-01 19:00:00  0.372  0.375   0.87  Lag   
5    SMR03-0219-0252 2019-05-01 18:00:00  0.000  0.000          --   
6    SMR03-0219-0252 2019-05-01 17:00:00  0.000  0.000          --   
7    SMR03-0219-0252 2019-05-01 16:00:00  0.000  0.000          --   
8    SMR03-0219-0252 2019-05-01 15:00:00  0.000  0.000          --   
9    SMR03-0219-0252 2019-05-01 14:00:00  0.000  0.000          --   
10   SMR03-0219-0252 2019-05-01 13:00:00  0.000  0.000          --   
11   SMR03-0219-0252 2019-05-01 12:00:00  0.000  0.000          --   
12   SMR03-0219-0252 2019-05-01 11:00:00  0.000  0.000          --   
13   SMR03-0219-0252

## Zero and non-zero dataframe

### Dataframe with off-time values (excluding faulty values)

In [10]:
off_time_df = df[df.time_hour >= 8]
off_time_df = off_time_df[off_time_df.time_hour <= 17]
off_time_df.describe()

,kwh,kvah,total_load,time_hour
count,308.0,308.0,3.080000e+02,308.000000
mean,0.0,0.0,5.170000e+00,12.519481
std,0.0,0.0,1.423398e-14,2.875757
min,0.0,0.0,5.170000e+00,8.000000
25%,0.0,0.0,5.170000e+00,10.000000
50%,0.0,0.0,5.170000e+00,13.000000
75%,0.0,0.0,5.170000e+00,15.000000
max,0.0,0.0,5.170000e+00,17.000000


### Dataframe with non-zero values (including faulty values)

In [11]:
on_time_df = df[(df.time_hour >= 18) | (df.time_hour <= 7)]
on_time_df.head()

,CCMS,timestamp,kwh,kvah,pf_total,total_load,time_hour
0,SMR03-0219-0252,2019-05-01 23:00:00,2.732,2.752,0.955 Ld,5.17,23
1,SMR03-0219-0252,2019-05-01 22:00:00,5.108,5.143,0.987 Ld,5.17,22
2,SMR03-0219-0252,2019-05-01 21:00:00,5.120,5.156,0.987 Ld,5.17,21
3,SMR03-0219-0252,2019-05-01 20:00:00,4.985,5.025,0.985 Ld,5.17,20
4,SMR03-0219-0252,2019-05-01 19:00:00,0.372,0.375,0.87 Lag,5.17,19


## Multiple analysis (sub analysis)
- time period when the street lights are off (plot using bar chart)
- random fluctuations of streetlights due to various reason like powercut etc (plot using scatter plot)
- streetlights pulling load lesser than the mean threshhold value (average KW/hr consumption +- std deviation)

### time period when lights where off

In [12]:
faulty_values = on_time_df[(on_time_df.kwh == 0) & (on_time_df.time_hour != 18) & (on_time_df.time_hour != 7)]
faulty_values

,CCMS,timestamp,kwh,kvah,pf_total,total_load,time_hour
355,SMR03-0219-0252,2019-04-17 02:00:00,0.0,0.0,--,5.17,2
356,SMR03-0219-0252,2019-04-17 01:00:00,0.0,0.0,--,5.17,1
379,SMR03-0219-0252,2019-04-16 02:00:00,0.0,0.0,--,5.17,2


In [13]:
faulty_timestamp = []
faulty_index = []
for index,row in on_time_df.iterrows():
    if row.kwh <= on_time_df.kwh.mean() - on_time_df.kwh.std() and row.kwh != 0:
        faulty_index.append(index)

In [14]:
for i in faulty_index:
    if df.iloc[i-1].kwh != 0 and df.iloc[i+1].kwh != 0:
        faulty_timestamp.append(df.iloc[i].timestamp)

In [15]:
i = 0
for index,row in df.iterrows():
    if row.timestamp == faulty_timestamp[i]:
        print(row)
        i = i + 1
        if i== len(faulty_timestamp):
            break

CCMS              SMR03-0219-0252
timestamp     2019-04-27 19:00:00
kwh                         0.561
kvah                        0.565
pf_total               0.795  Lag
total_load                   5.17
time_hour                      19
Name: 100, dtype: object
CCMS              SMR03-0219-0252
timestamp     2019-04-15 00:00:00
kwh                          0.13
kvah                         0.13
pf_total                0.786  Ld
total_load                   5.17
time_hour                       0
Name: 405, dtype: object
CCMS              SMR03-0219-0252
timestamp     2019-04-14 23:00:00
kwh                         0.819
kvah                        0.824
pf_total                0.988  Ld
total_load                   5.17
time_hour                      23
Name: 406, dtype: object


# model

In [17]:
model = load_model('sl_classifier.h5')
df_array = on_time_df.kwh

In [18]:
df_array = np.array(df_array)
predicted_values = model.predict(df_array)

In [19]:
on_time_classified = on_time_df.reset_index()

In [20]:
on_time_classified['predicted_values'] = predicted_values

In [23]:
on_time_classified.head(10)

,index,CCMS,timestamp,kwh,kvah,pf_total,total_load,time_hour,predicted_values
0,0,SMR03-0219-0252,2019-05-01 23:00:00,2.732,2.752,0.955 Ld,5.17,23,0.520734
1,1,SMR03-0219-0252,2019-05-01 22:00:00,5.108,5.143,0.987 Ld,5.17,22,0.999981
2,2,SMR03-0219-0252,2019-05-01 21:00:00,5.120,5.156,0.987 Ld,5.17,21,0.999982
3,3,SMR03-0219-0252,2019-05-01 20:00:00,4.985,5.025,0.985 Ld,5.17,20,0.999972
4,4,SMR03-0219-0252,2019-05-01 19:00:00,0.372,0.375,0.87 Lag,5.17,19,0.002439
5,5,SMR03-0219-0252,2019-05-01 18:00:00,0.000,0.000,--,5.17,18,0.002296
6,16,SMR03-0219-0252,2019-05-01 07:00:00,0.000,0.000,--,5.17,7,0.002296
7,17,SMR03-0219-0252,2019-05-01 06:00:00,3.355,3.387,0.387 Ld,5.17,6,0.993839
8,18,SMR03-0219-0252,2019-05-01 05:00:00,5.052,5.095,0.981 Ld,5.17,5,0.999978
9,19,SMR03-0219-0252,2019-05-01 04:00:00,5.156,5.194,0.981 Ld,5.17,4,0.999984
